# 머신러닝을 이용한 투자전략 : ETF상승하락예측

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

# 밑에 실행하면서 뜨는 빨간 경고 팝업 뜨지 않게 하기 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib   # 한국어 출력되게 
import os
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)
import plotly.graph_objs as go

In [3]:
data = pd.read_csv("./data/ETFs_main.csv")
data

,Dates,CLOSE_SPY,OPEN,HIGH,LOW,VOLUME,CLOSE_GLD,CLOSE_FXY,CLOSE_T10Y2Y,CLOSE_TED,CLOSE_USO,CLOSE_UUP,CLOSE_VIX,CLOSE_VWO
0,2007-02-20,146.04,145.56,146.200,144.0,56909500.0,65.31,83.51,2.3263,0.31,48.67,25.07,10.24,40.055
1,2007-02-21,145.98,145.61,146.070,145.0,63971500.0,67.28,82.90,2.3653,0.32,49.86,25.12,10.20,39.975
2,2007-02-22,145.87,146.05,146.420,145.0,79067398.0,67.15,82.46,2.3871,0.31,50.33,25.12,10.18,40.220
3,2007-02-23,145.30,145.74,145.790,145.0,71962797.0,67.72,82.78,2.3809,0.31,50.46,25.04,10.58,40.035
4,2007-02-26,145.17,145.83,145.950,145.0,69320062.0,68.10,83.08,2.3795,0.31,50.90,25.04,11.15,39.960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2766,2018-12-20,247.17,249.86,251.620,245.0,252053406.0,119.24,85.87,1.7807,0.48,9.72,25.77,28.38,38.180
2767,2018-12-21,240.70,246.74,249.710,240.0,255345594.0,118.72,85.87,1.7651,0.48,9.57,25.94,30.11,37.870
2768,2018-12-24,234.34,239.04,240.836,234.0,147311594.0,120.02,86.55,1.7505,0.40,9.29,25.55,36.07,37.320
2769,2018-12-27,248.07,242.57,248.290,239.0,186267297.0,120.57,86.00,1.7581,0.44,9.62,25.57,29.96,37.900


In [4]:
data.columns

Index(['Dates', 'CLOSE_SPY', 'OPEN', 'HIGH', 'LOW', 'VOLUME', 'CLOSE_GLD',
       'CLOSE_FXY', 'CLOSE_T10Y2Y', 'CLOSE_TED', 'CLOSE_USO', 'CLOSE_UUP',
       'CLOSE_VIX', 'CLOSE_VWO'],
      dtype='object')

## 칼럼 설명

- CLOSE_SPY: SPY(S&P 500 ETF)의 종가. SPY는 S&P 500 지수를 추종하는 ETF로, 미국 주식시장의 전반적인 성과를 반영합니다.
- 추가 설명: SPY는 미국의 대표적인 주가지수 ETF로, S&P 500에 속한 500대 대형주들의 가격 변동을 추적합니다. 투자자들이 미국 경제 전반에 대한 노출을 원할 때 많이 사용하는 대표적인 ETF입니다.
<br><br>
- OPEN: SPY의 시가. 거래가 시작될 때의 가격을 의미합니다.
- 추가 설명: 시가는 거래일의 첫 거래가 이루어지는 순간의 가격입니다. 시가와 종가의 차이는 주가가 하루 동안 어떤 방향으로 변동했는지를 알 수 있게 해줍니다.
<br><br>
- HIGH: SPY의 고가. 해당 거래일 동안 기록된 가장 높은 가격입니다.
- 추가 설명: 고가는 특정 거래일 동안의 최대 가격을 의미하며, 투자자들의 수요가 크게 증가했을 때 기록됩니다. 이는 시장의 관심과 구매가 강했음을 보여줍니다.
<br><br>
- LOW: SPY의 저가. 해당 거래일 동안 기록된 가장 낮은 가격입니다.
- 추가 설명: 저가는 해당 거래일 동안 기록된 최저 가격으로, 주식이 가장 약세를 보였던 시점을 나타냅니다. 저가와 고가의 차이는 변동성의 크기를 가늠하는 데 유용합니다.
<br><br>
- VOLUME: SPY의 거래량. 해당 거래일 동안 주식이 얼마나 많이 거래되었는지를 나타냅니다.
- 추가 설명: 거래량은 해당 주식의 유동성과 시장 참여자의 관심을 보여줍니다. 거래량이 높다면 해당 주식에 대한 관심이 높고, 거래량이 적다면 상대적으로 거래가 활발하지 않다는 의미입니다.
<br><br>
- CLOSE_GLD: GLD(Gold ETF)의 종가. 금 가격을 추종하는 ETF로, 금의 시장 가격을 반영합니다.
- 추가 설명: GLD는 금에 투자하고 싶지만 실물 금을 보유하지 않고 금 가격에 따라 움직이는 ETF를 통해 간접적으로 투자할 수 있는 방법입니다. 금은 안전자산으로 인식되기 때문에, 글로벌 경제 위기나 불확실성이 있을 때 수요가 증가하는 경향이 있습니다.
<br><br>
- CLOSE_FXY: FXY(Japanese Yen ETF)의 종가. 일본 엔화의 성과를 추종하는 ETF입니다.
- 추가 설명: FXY는 일본 엔화의 환율 변동을 추종합니다. 엔화는 국제 금융시장에서 안전자산으로 여겨지며, 글로벌 시장의 불확실성이 커질 때 엔화의 가치는 상승하는 경향이 있습니다.
<br><br>
- CLOSE_T10Y2Y: 10년 만기 미국 국채와 2년 만기 국채 간의 금리 차이를 나타내는 지표입니다. 일반적으로 경기 예측에 사용됩니다.
- 추가 설명: 이 금리 차이(일명 수익률 곡선)는 경기 침체의 예측 도구로 자주 사용됩니다. 일반적으로 장기 국채(10년)의 금리가 단기 국채(2년)보다 높지만, 역전이 발생하면 경기 침체 가능성이 있다고 해석됩니다.
<br><br>
- CLOSE_TED: TED 스프레드(TED Spread)로, 미국 3개월 만기 재무부채권 금리와 3개월 만기 유로달러 금리 차이를 나타냅니다. 금융시장의 위험을 측정하는 지표로 자주 사용됩니다.
- 추가 설명: TED 스프레드는 금융 위기가 다가올 때 급격히 상승하는 경향이 있습니다. 스프레드가 높아지면 금융기관 간의 신용 리스크가 높아지고, 자금 시장에서 불안정성이 증가한 것으로 해석됩니다.
<br><br>
- CLOSE_USO: USO(Crude Oil ETF)의 종가. 원유 가격을 추종하는 ETF입니다.
- 추가 설명: USO는 원유 가격에 투자하고자 하는 투자자들에게 인기가 있는 ETF입니다. 원유는 전 세계 경제에 중요한 역할을 하는 자원으로, 원유 가격은 공급과 수요, 지정학적 리스크 등에 민감하게 반응합니다.
<br><br>
- CLOSE_UUP: UUP(U.S. Dollar ETF)의 종가. 미국 달러 지수를 추종하는 ETF로, 달러의 가치 변동을 반영합니다.
- 추가 설명: UUP는 미국 달러의 강세 또는 약세에 투자할 수 있는 수단을 제공합니다. 국제 금융 시장에서 달러는 기축 통화로써 중요한 역할을 하며, 글로벌 경제 상황에 따라 달러의 가치가 변동합니다.
<br><br>
- CLOSE_VIX: VIX(Volatility Index) 종가. 흔히 '공포 지수'라고도 불리며, 시장의 변동성(주로 S&P 500의 향후 30일간의 예상 변동성)을 나타냅니다.
- 추가 설명: VIX는 시장의 변동성을 측정하는 지표로, VIX 지수가 높으면 시장이 불안정하고 투자자들이 리스크를 회피하려는 경향이 강하다는 신호입니다. VIX 지수는 20 이하일 때 상대적으로 안정된 시장을 의미하며, 30 이상일 때는 높은 변동성을 예상합니다.
<br><br>
- CLOSE_VWO: VWO(Emerging Markets ETF)의 종가. 신흥 시장 주식에 투자하는 ETF입니다.
- 추가 설명: VWO는 중국, 인도, 브라질 등 신흥 시장 경제에 대한 노출을 제공합니다. 신흥 시장은 성장 잠재력이 크지만 변동성 또한 높아, 투자자들은 리스크와 보상의 균형을 고려해야 합니다.
<br><br>

## 분석에 사용할 기술적 지표 만들기
- MA_45: 45일 단순 이동평균(Simple Moving Average): 특정 자산의 45일간 평균 증가

- VMA_45: 45일 거래량(Volume Moving Average): 지난 45일간의 평균 거래량

- RSI_14: 14일 상대강도지수(Relative Strength Index): 14일 동안의 자산 가격 변동을 바탕으로 과매수 또는 과매도 상태를 평가하는 지표


## RSI(Relative Strength Index) 공식

* RSI는 자산의 가격 변동 강도를 측정해 과매수, 또는 과매도 상태를 평가하는 기술적 분석 지표
* 주로 14일 동안의 가격 변동 기준으로 계산, 값은 0 - 100 사이
* RSI 값이 70 이상: 과매수 상태(매도 시점을 고려)
* RSI 값이 30 이하: 과매도 상태(매수 시점을 고려)
* RSI는 주가가 너무 빠르게 상승하거나 하락했는지 확인하는 데 사용
* 투자자들의 매수/매도 결정을 내릴 때 중요한 참고 지표로 활용
<br><br>

### RSI 계산 공식
$$ RSI = \left( 100 - \frac{100}{1 + RS} \right) $$
### RS(Relative Strength)
$$ RS = \frac{Average\ Gain}{Average\ Loss} $$
* Average Gain: 일정 기간(14일) 동안 가격 상승분의 평균
* Average Loss: 일정 기간(14일) 동안 가격 하락분의 평균
* 상승 Gain = 오늘의 종가 - 어제의 종가 
* 하락 Loss = 어제의 종가 - 오늘의 종가 

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2771 entries, 0 to 2770
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Dates         2771 non-null   object 
 1   CLOSE_SPY     2771 non-null   float64
 2   OPEN          2771 non-null   float64
 3   HIGH          2771 non-null   float64
 4   LOW           2771 non-null   float64
 5   VOLUME        2771 non-null   float64
 6   CLOSE_GLD     2771 non-null   float64
 7   CLOSE_FXY     2771 non-null   float64
 8   CLOSE_T10Y2Y  2771 non-null   float64
 9   CLOSE_TED     2771 non-null   float64
 10  CLOSE_USO     2771 non-null   float64
 11  CLOSE_UUP     2771 non-null   float64
 12  CLOSE_VIX     2771 non-null   float64
 13  CLOSE_VWO     2771 non-null   float64
dtypes: float64(13), object(1)
memory usage: 303.2+ KB


In [6]:
data['Dates'] = pd.to_datetime(data['Dates'])
data['Dates'].dtype

dtype('<M8[ns]')

dtype('<M8[ns]') : 데이트 date 타입을 의미함

In [ ]:
# 거래량 이동평균 vma 
vma = 

In [ ]:
data = data.join(vma)
data